# Advent of Code
## Day 7: Amplification Circuit
https://adventofcode.com/2019/day/7

----
## Part 1
----

In [1]:
import itertools
import copy
from collections import deque

In [2]:
def opcode_to_str(opcode_int):
    '''
    Take integer opcode and convert it to string of length 5, with left padded 0s
    '''
    opcode_len = len(str(opcode_int)) 
    return (5 - opcode_len) * '0' + str(opcode_int)

In [46]:
def run_intcode(int_input, program):
    
    position = 0
    output = []
    opcode = ''
    
    while True:
        
        opcode = opcode_to_str(program[position])                
            
        # opcode '01' is addition, takes 3 parameters
        if opcode[-2:] == '01':
            
            if opcode[-3] == '0':
                param_1 = program[program[position + 1]]
            elif opcode[-3] == '1':
                param_1 = program[position + 1]
                
            if opcode[-4] == '0':
                param_2 = program[program[position + 2]]
            elif opcode[-4] == '1':
                param_2 = program[position + 2]
            
            # Controls WHERE the values are written
            if opcode[-5] == '0':
                program[program[position + 3]] = param_1 + param_2
            elif opcode[-5] == '1':
                program[position + 3] = param_1 + param_2
            
            # Move to next "opcode"
            position += 4 
            

        # opcode '02' is multiplication, takes 3 parameters
        elif opcode[-2:] == '02':

            if opcode[-3] == '0':
                param_1 = program[program[position + 1]]
            elif opcode[-3] == '1':
                param_1 = program[position + 1]
                
            if opcode[-4] == '0':
                param_2 = program[program[position + 2]]
            elif opcode[-4] == '1':
                param_2 = program[position + 2]
            
            
            # Controls WHERE the values are written
            if opcode[-5] == '0':
                program[program[position + 3]] = param_1 * param_2
            elif opcode[-5] == '1':
                program[position + 3] = param_1 * param_2

            # Move to next "opcode"
            position += 4 


        # opcode '03' is input, takes 1 parameter
        elif opcode[-2:] == '03':  
            
            program[program[position + 1]] = int_input.pop(0)

            position += 2 # Move to next "opcode"

            
        # opcode '04' is output, takes 1 parameter
        elif opcode[-2:] == '04':
            
            if opcode[-3] == '0':
                param_1 = program[program[position + 1]]
            elif opcode[-3] == '1':
                param_1 = program[position + 1]
            
            output.append(param_1)

            position += 2 # Move to next "opcode"

            
        # opcode '05', jump if true
        elif opcode[-2:] == '05':
            
            if opcode[-3] == '0':
                param_1 = program[program[position + 1]]
            elif opcode[-3] == '1':
                param_1 = program[position + 1]

            if opcode[-4] == '0':
                param_2 = program[program[position + 2]]
            elif opcode[-4] == '1':
                param_2 = program[position + 2]
            
            if param_1 != 0:
                position = param_2
            else:
                position += 3
                
        
        # opcode '06', jump if false
        elif opcode[-2:] == '06':

            if opcode[-3] == '0':
                param_1 = program[program[position + 1]]
            elif opcode[-3] == '1':
                param_1 = program[position + 1]

            if opcode[-4] == '0':
                param_2 = program[program[position + 2]]
            elif opcode[-4] == '1':
                param_2 = program[position + 2]
            
            if param_1 == 0:
                position = param_2
            else:
                position += 3
        
        
        # opcode '07', less than
        elif opcode[-2:] == '07':

            if opcode[-3] == '0':
                param_1 = program[program[position + 1]]
            elif opcode[-3] == '1':
                param_1 = program[position + 1]

            if opcode[-4] == '0':
                param_2 = program[program[position + 2]]
            elif opcode[-4] == '1':
                param_2 = program[position + 2]
                
            # Controls WHERE the values are written
            if opcode[-5] == '0':
                if param_1 < param_2:
                    program[program[position + 3]] = 1
                else:
                    program[program[position + 3]] = 0                
            
            elif opcode[-5] == '1':
                if param_1 < param_2:
                    program[position + 3] = 1
                else:
                    program[position + 3] = 0

            position += 4
        
        
        # opcode '08', equals
        elif opcode[-2:] == '08':

            if opcode[-3] == '0':
                param_1 = program[program[position + 1]]
            elif opcode[-3] == '1':
                param_1 = program[position + 1]

            if opcode[-4] == '0':
                param_2 = program[program[position + 2]]
            elif opcode[-4] == '1':
                param_2 = program[position + 2]
                
            # Controls WHERE the values are written
            if opcode[-5] == '0':
                if param_1 == param_2:
                    program[program[position + 3]] = 1
                else:
                    program[program[position + 3]] = 0                
            
            elif opcode[-5] == '1':
                if param_1 == param_2:
                    program[position + 3] = 1
                else:
                    program[position + 3] = 0

            position += 4                    

        # opcode '99' is terminate, takes 0 parameters    
        elif opcode[-2:] == '99':
            break
            
    return output[0]

In [51]:
def run_amps(phase_settings, program):
    """
    Run intcode programs in sequence with a set of phase settings and
    output from the previous program.
    """
    
    q = deque([0]) # Start off with 0 as the first signal sent into Amp A 

    for phase in phase_settings:
        
        # Each amplifier runs an unaltered version of the program
        program_copy = copy.deepcopy(program)
        
        # Populate the que
        q.appendleft(phase)
        q.appendleft(run_intcode(list(q), program_copy))    
        
        # Clear the que
        signal = q.pop()
        phase = q.pop()
    
    last_signal = q
        
    return last_signal[0]

## Load some test data

In [52]:
test_phases = [(4,3,2,1,0),
               (0,1,2,3,4),
               (1,0,4,3,2)]

In [53]:
test_programs = [[3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0],
                 [3,23,3,24,1002,24,10,24,1002,23,-1,23,101,5,23,23,1,24,23,23,4,23,99,0,0],
                 [3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0]]

## Run some tests

In [54]:
run_amps(test_phases[0], test_programs[0])

43210

In [55]:
run_amps(test_phases[1], test_programs[1])

54321

In [56]:
run_amps(test_phases[2], test_programs[2])

65210

## Load program (puzzle input)

In [57]:
with open("inputs\\amplifier_control_software.txt") as file:
    for line in file:
        program = [int(x) for x in line.split(',')]        

## Create all possible permutations of phase settings

In [58]:
phase_sets = itertools.permutations(range(5))

## Run all possible permutations through the amps and get the max(thrust)

In [59]:
%%time
thrusts = [run_amps(phase, program) for phase in phase_sets]
print(max(thrusts))

567045
Wall time: 257 ms


# Bingo!

----
## Part 2
----

### Create `int_computer` as a generator function

In [117]:
def run_intcode_gen(int_input, program):
    """
    Generator function version of the intcode computer
    """
    position = 0
    output = []
    opcode = ''
    program = copy.deepcopy(program)
    
    while True:
        
        opcode = opcode_to_str(program[position])                
            
        # opcode '01' is addition, takes 3 parameters
        if opcode[-2:] == '01':
            
            if opcode[-3] == '0':
                param_1 = program[program[position + 1]]
            elif opcode[-3] == '1':
                param_1 = program[position + 1]
                
            if opcode[-4] == '0':
                param_2 = program[program[position + 2]]
            elif opcode[-4] == '1':
                param_2 = program[position + 2]
            
            # Controls WHERE the values are written
            if opcode[-5] == '0':
                program[program[position + 3]] = param_1 + param_2
            elif opcode[-5] == '1':
                program[position + 3] = param_1 + param_2
            
            # Move to next "opcode"
            position += 4 
            

        # opcode '02' is multiplication, takes 3 parameters
        elif opcode[-2:] == '02':

            if opcode[-3] == '0':
                param_1 = program[program[position + 1]]
            elif opcode[-3] == '1':
                param_1 = program[position + 1]
                
            if opcode[-4] == '0':
                param_2 = program[program[position + 2]]
            elif opcode[-4] == '1':
                param_2 = program[position + 2]
            
            
            # Controls WHERE the values are written
            if opcode[-5] == '0':
                program[program[position + 3]] = param_1 * param_2
            elif opcode[-5] == '1':
                program[position + 3] = param_1 * param_2

            # Move to next "opcode"
            position += 4 


        # opcode '03' is input, takes 1 parameter
        elif opcode[-2:] == '03':  
            
            program[program[position + 1]] = int_input.pop(0)

            position += 2 # Move to next "opcode"

            
        # opcode '04' is output, takes 1 parameter
        elif opcode[-2:] == '04':
            
            if opcode[-3] == '0':
                param_1 = program[program[position + 1]]
            elif opcode[-3] == '1':
                param_1 = program[position + 1]

            #output.append(param_1)
            yield param_1  # Makes function a generator!!!
            
            # Move to next "opcode"
            position += 2 

            
        # opcode '05', jump if true
        elif opcode[-2:] == '05':
            
            if opcode[-3] == '0':
                param_1 = program[program[position + 1]]
            elif opcode[-3] == '1':
                param_1 = program[position + 1]

            if opcode[-4] == '0':
                param_2 = program[program[position + 2]]
            elif opcode[-4] == '1':
                param_2 = program[position + 2]
            
            # Move to next "opcode"
            if param_1 != 0:
                position = param_2
            else:
                position += 3
                
        
        # opcode '06', jump if false
        elif opcode[-2:] == '06':

            if opcode[-3] == '0':
                param_1 = program[program[position + 1]]
            elif opcode[-3] == '1':
                param_1 = program[position + 1]

            if opcode[-4] == '0':
                param_2 = program[program[position + 2]]
            elif opcode[-4] == '1':
                param_2 = program[position + 2]
            
            # Move to next "opcode"
            if param_1 == 0:
                position = param_2
            else:
                position += 3
        
        
        # opcode '07', less than
        elif opcode[-2:] == '07':

            if opcode[-3] == '0':
                param_1 = program[program[position + 1]]
            elif opcode[-3] == '1':
                param_1 = program[position + 1]

            if opcode[-4] == '0':
                param_2 = program[program[position + 2]]
            elif opcode[-4] == '1':
                param_2 = program[position + 2]
                
            # Controls WHERE the values are written
            if opcode[-5] == '0':
                if param_1 < param_2:
                    program[program[position + 3]] = 1
                else:
                    program[program[position + 3]] = 0                
            
            elif opcode[-5] == '1':
                if param_1 < param_2:
                    program[position + 3] = 1
                else:
                    program[position + 3] = 0

            # Move to next "opcode"
            position += 4
        
        
        # opcode '08', equals
        elif opcode[-2:] == '08':

            if opcode[-3] == '0':
                param_1 = program[program[position + 1]]
            elif opcode[-3] == '1':
                param_1 = program[position + 1]

            if opcode[-4] == '0':
                param_2 = program[program[position + 2]]
            elif opcode[-4] == '1':
                param_2 = program[position + 2]
                
            # Controls WHERE the values are written
            if opcode[-5] == '0':
                if param_1 == param_2:
                    program[program[position + 3]] = 1
                else:
                    program[program[position + 3]] = 0                
            
            elif opcode[-5] == '1':
                if param_1 == param_2:
                    program[position + 3] = 1
                else:
                    program[position + 3] = 0

            position += 4                    

            
        # opcode '99' is terminate, takes 0 parameters    
        elif opcode[-2:] == '99':
            break
            

In [131]:
def run_amplifiers(phases, program):
    """
    Source: https://twitter.com/FogleBird/status/1203366610020556806/photo/1
    """
    
    # Convert phases to a list of lists, as a list will be passed as inputs to the 
    # int_computer_gen function
    inputs = [[phase] for phase in phases]
    
    # Append a 0 to the first element, as this is the initial input
    inputs[0].append(0)
    
    #print(inputs)
    
    # Create a list of running amplifiers
    # len(inputs) = 5 -- one for each phase setting
    # The first input has two elements in the list, but all the
    # rest have just the phase settings
    amplifiers = [run_intcode_gen(i, program) for i in inputs]
    
    #print(amplifiers) # produces a list of generator objects
    
    # Start an infinite feedback-loop
    while True:
        
        # For each loop in the feedback-loop, run through all 5 amplifiers
        for i, amp in enumerate(amplifiers):
            
            #print(inputs)
            
            try:
                signal = next(amp) # next(amp) makes next function call and collects the signal
            
            # Not sure what StopIteration is/does...
            except StopIteration:
                return signal # spits out the last signal, to be sent to the Thruster
            
            # Clever: (i + 1) % len(inputs) cycles through 0:4 as i increases
            inputs[(i + 1) % len(inputs)].append(signal)

## Test Data

In [132]:
test_phases_2 = [(9,8,7,6,5),
                 (9,7,8,5,6)]

In [133]:
test_programs_2 = [[3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5],
                   [3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,-5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10]]

In [134]:
run_amplifiers(test_phases_2[0], test_programs_2[0])

139629729

In [135]:
run_amplifiers(test_phases_2[1], test_programs_2[1])

18216

In [149]:
phase_sets = itertools.permutations(range(5,10))

In [169]:
max(run_amplifiers(phases, program) for phases in phase_sets)

39016654

## That was hard!